### Gemma

In [1]:
from transformers import GemmaForCausalLM,GemmaTokenizer,BitsAndBytesConfig
import torch
from peft import LoraConfig,get_peft_model,prepare_model_for_kbit_training

/home/zzzzz/miniconda3/envs/vsr_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
)
tokenizer = GemmaTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model = GemmaForCausalLM.from_pretrained("google/gemma-1.1-2b-it",quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


In [5]:
prompt = "Il gatto"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
input_ids = input_ids.to(model.device)
print(input_ids)
output = model(input_ids, return_dict=True)

tensor([[     2,   6870, 157593]], device='cuda:0')


In [6]:
print(output.logits.shape)

torch.Size([1, 3, 256000])


In [ ]:
special_tokens_dict = ["<STT>"]
num_added_toks = tokenizer.add_tokens(special_tokens_dict,special_tokens=True)
print("We have added", num_added_toks, "tokens")
model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenizer.special_tokens_map

In [ ]:
prompt = "I know <STT> is a good person."
input_ids = tokenizer.encode(prompt, return_tensors="pt",skip_special_tokens=True)
print(tokenizer.decode(input_ids[0],skip_special_tokens=True))

: 

In [ ]:
prompt = ["I like deep learning", "I like to play with my dog", "I like to play with my cat", "I like to play with my"]
tokenizer.padding_side = "left"
tokens = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
mask=tokens.attention_mask

In [ ]:
print(mask)
mask.sum(-1)

In [ ]:
input = tokenizer("You can generate the output not include the input?",add_special_tokens=True,return_tensors='pt')

In [ ]:
tokenizer.batch_decode(input["input_ids"],skip_special_tokens=False)

In [ ]:
fwd = model(**input,return_dict=True)

In [ ]:
fwd.logits
torch.argmax(torch.softmax(fwd.logits,dim=-1),dim=-1)

In [ ]:
tokenizer.batch_decode(torch.argmax(torch.softmax(fwd.logits,dim=-1),dim=-1),skip_special_tokens=False)

In [ ]:
gt = model.generate(input["input_ids"],do_sample=True,max_length=100,num_return_sequences=1,top_k=50,top_p=0.95,temperature=0.9)

In [ ]:
tokenizer.decode(gt[0],skip_special_tokens=False)

In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
a = torch.tensor([4,5,6])
b = torch.tensor([1,2])
pad_sequence([a,b],batch_first=True,padding_value=0)

tensor([[4, 5, 6],
        [1, 2, 0]])

In [6]:
a = torch.tensor([[4,5,6],[1,2,0]])
print(a.shape)

torch.Size([2, 3])


In [1]:
from tensorboardX import SummaryWriter

In [2]:
writer = SummaryWriter("logs")
writer.add_scalar("test",0.5)
writer.add_scalar("test",0.6)
writer.close()

In [4]:
!tensorboard --logdir logs --port 6007

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.17.0 at http://localhost:6007/ (Press CTRL+C to quit)
^C


### OPT

In [14]:
import os
import torch
os.environ["HF_HOME"] = "/home/zzzzz/.cache/huggingface/hub"
from transformers import OPTForCausalLM, GPT2Tokenizer
model = OPTForCausalLM.from_pretrained("facebook/opt-350m", cache_dir="/home/zzzzz/.cache/huggingface/hub/models--facebook--opt-350m")
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-350m",cache_dir="/home/zzzzz/.cache/huggingface/hub/models--facebook--opt-350m")
embedding = model.get_input_embeddings()

Embedding(50272, 512, padding_idx=1)


/home/zzzzz/miniconda3/envs/vsr_llm_avsr/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
for name, param in model.named_parameters():
    print(name, param.shape)

model.decoder.embed_tokens.weight torch.Size([50272, 512])
model.decoder.embed_positions.weight torch.Size([2050, 1024])
model.decoder.project_out.weight torch.Size([512, 1024])
model.decoder.project_in.weight torch.Size([1024, 512])
model.decoder.layers.0.self_attn.k_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.k_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn.v_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.v_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn.q_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.q_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.out_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn_layer_norm.weight torch.Size([1024])
model.decoder.layers.0.self_attn_layer_norm.bias torch.Size([1024])
model.decoder.layers.0.fc1.weight torch.Size([4096, 1024])
model.decoder.layers.

In [20]:
chat = "I like to play with my dog."
tokens = tokenizer.tokenize(chat)
print(tokens)

['I', 'Ġlike', 'Ġto', 'Ġplay', 'Ġwith', 'Ġmy', 'Ġdog', '.']


In [26]:
print(tokenizer.special_tokens_map)
tokenizer.bos_token_id

{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '<pad>'}


2

In [21]:
tokenizer.decode(tokenizer.encode(chat))

'</s>I like to play with my dog.'

In [5]:
import torch
a = torch.ones(2,3).fill_(-100)

In [4]:
import torch
import os
path ="/home/zzzzz/auto_avsr/data/LRS2_preprocess/labels/lrs2_test_transcript_lengths_seg24s.csv"
data_list = []
with open(os.path.join(path)) as f:
    data_list.extend(map(lambda x:(x.split(",")[0],x.split(",")[1],int(x.split(",")[2]),torch.tensor([int(_) for _ in x.split(",")[3].split()])),f.read().splitlines()))

In [5]:
data_list

[('lrs2',
  'lrs2_video_seg24s/main/6330311066473698535/00011.mp4',
  62,
  tensor([ 713, 2176, 3052, 4575, 4464, 4891])),
 ('lrs2',
  'lrs2_video_seg24s/main/6330311066473698535/00018.mp4',
  27,
  tensor([4590,    2,  393, 3183,  777])),
 ('lrs2',
  'lrs2_video_seg24s/main/6330311066473698535/00022.mp4',
  26,
  tensor([ 713, 4903, 4923, 3981])),
 ('lrs2',
  'lrs2_video_seg24s/main/6330311066473698535/00025.mp4',
  22,
  tensor([ 713, 4575, 3258, 1530])),
 ('lrs2',
  'lrs2_video_seg24s/main/6331559613336179781/00019.mp4',
  45,
  tensor([4934, 4587, 2737,    2,  425, 3187, 1838, 2598, 4575, 2262])),
 ('lrs2',
  'lrs2_video_seg24s/main/6331559613336179781/00020.mp4',
  42,
  tensor([4280, 3327, 4092, 2405, 3327, 3325])),
 ('lrs2',
  'lrs2_video_seg24s/main/6331559613336179781/00021.mp4',
  33,
  tensor([4921, 2707, 3290])),
 ('lrs2',
  'lrs2_video_seg24s/main/6331559613336179781/00027.mp4',
  46,
  tensor([4903, 1085, 2782,  549, 2297,  850, 4969, 2743, 3291])),
 ('lrs2',
  'lrs2_vide

## Load front encoder

In [3]:
path = "/home/zzzzz/VSR_LLM_AVSR/ckpts/auto_avsr/vsr_trlrwlrs2lrs3vox2avsp_base.pth"

In [4]:
state_dict = torch.load(path)

/tmp/ipykernel_21610/1018981882.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path)
